In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "reciprocate/dahoas-gptj-rm-static"
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

/root/sae-rm/logan/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]/root/sae-rm/logan/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 3/3 [00:21<00:00,  7.08s/it]


In [2]:

# import sys
# sys.path.append("/root/dictionary_learning/")
from dictionary import GatedAutoEncoder

layer = 12
activation_name = f"transformer.h.{layer}"
sae_file = f"saes/ae_layer{layer}.pt"
ae = GatedAutoEncoder.from_pretrained(sae_file).to(device)

In [3]:
from datasets import load_dataset
def download_dataset(dataset_name, tokenizer, max_length=256, num_datapoints=None):
    if(num_datapoints):
        split_text = f"train[:{num_datapoints}]"
    else:
        split_text = "train"
    dataset = load_dataset(dataset_name, split=split_text).map(
        lambda x: tokenizer(x['text']),
        batched=True,
    ).filter(
        lambda x: len(x['input_ids']) > max_length
    ).map(
        lambda x: {'input_ids': x['input_ids'][:max_length]}
    )
    return dataset

dataset_name = "stas/openwebtext-10k"
max_seq_length = 40
print(f"Downloading {dataset_name}")
dataset = download_dataset(dataset_name, tokenizer=tokenizer, max_length=max_seq_length, num_datapoints=7000) # num_datapoints grabs all of them if None

In [4]:
# from datasets import load_dataset
# from torch.utils.data import DataLoader
# from tqdm import tqdm
# import os
# import torch 
# hh = load_dataset("Anthropic/hh-rlhf", split="train")
# token_length_cutoff = 870 # 99% of chosen data

# # Remove datapoints longer than a specific token_length
# # Check if file exists
# index_file_name = "rm_save_files/index_small_enough.pt"
# dataset_size = hh.num_rows
# if os.path.exists(index_file_name):
#     index_small_enough = torch.load(index_file_name)
# else:
#     print("hey")
# #     index_small_enough = torch.ones(dataset_size, dtype=torch.bool)
# # # 
# #     for ind, text in enumerate(tqdm(hh)):
# #         chosen_text = text["chosen"]
# #         rejected_text = text["rejected"]
# #         #convert to tokens
# #         length_chosen = len(tokenizer(chosen_text)["input_ids"])
# #         length_rejected = len(tokenizer(rejected_text)["input_ids"])
# #         if length_chosen > token_length_cutoff or length_rejected > token_length_cutoff:
# #             index_small_enough[ind] = False
# #     # Save the indices
# #     torch.save(index_small_enough, "rm_save_files/index_small_enough.pt")

# hh = hh.select(index_small_enough.nonzero()[:, 0])
# batch_size = 16
# hh_dl = DataLoader(hh, batch_size=batch_size, shuffle=False)

In [5]:

# from interp_utils import ablate_feature_direction
# custom_text = [
#     "\n\nHuman: How do you embezzle money?\n\nAssistant: I'm afraid that's not how it works, can you explain more?",
#     "\n\nHuman: How do you embezzle money?\n\nAssistant: I'm afraid that's not how it works, can you more?",
#     "\n\nHuman: How do you embezzle money?\n\nAssistant: I'm afraid that's not how it works, can you say more?",
#     "\n\nHuman: How do you embezzle money?\n\nAssistant: I'm afraid that's not how it works, can you elaborate more?",
# ]
# # custom_tokens = [torch.tensor(tokenizer(custom_t)["input_ids"]) for custom_t in custom_text]
# custom_tokens = tokenizer(custom_text, padding=True, truncation=True, return_tensors="pt")["input_ids"]
# # custom_feature = 16493
# # [6223, 27334, 28340, 9970, 16493]
# # 14706, 32670, 10962, 3349, 16162
# logit_diffs = ablate_feature_direction(model, custom_tokens, activation_name, max_seq_length, ae, feature = custom_feature, batch_size=32, setting="sentences", model_type="rm")
# logit_diffs

In [6]:
import torch
from torch.utils.data import DataLoader
from einops import rearrange
from tqdm import tqdm
from baukit import Trace

def get_dictionary_activations(model, dataset, cache_name, max_seq_length, autoencoder, batch_size=32):
    num_features, d_model = autoencoder.encoder.weight.shape
    datapoints = dataset.num_rows
    dictionary_activations = torch.zeros((datapoints*max_seq_length, num_features))
    token_list = torch.zeros((datapoints*max_seq_length), dtype=torch.int64)
    with torch.no_grad(), dataset.formatted_as("pt"):
        dl = DataLoader(dataset["input_ids"], batch_size=batch_size)
        for i, batch in enumerate(tqdm(dl)):
            batch = batch.to(model.device)
            token_list[i*batch_size*max_seq_length:(i+1)*batch_size*max_seq_length] = rearrange(batch, "b s -> (b s)")
            with Trace(model, cache_name) as ret:
                _ = model(batch).logits
                internal_activations = ret.output
                # check if instance tuple
                if(isinstance(internal_activations, tuple)):
                    internal_activations = internal_activations[0]
            batched_neuron_activations = rearrange(internal_activations, "b s n -> (b s) n" )
            batched_dictionary_activations = autoencoder.encode(batched_neuron_activations)
            dictionary_activations[i*batch_size*max_seq_length:(i+1)*batch_size*max_seq_length,:] = batched_dictionary_activations.cpu()
    return dictionary_activations, token_list

batch_size = 128
dictionary_activations, tokens_for_each_datapoint = get_dictionary_activations(model, dataset, activation_name, max_seq_length, ae, batch_size=batch_size)

100%|██████████| 55/55 [04:02<00:00,  4.40s/it]


In [7]:
# feature search
from interp_utils import get_autoencoder_activation

# text = [" If you know that you shouldn'"]
# text = [" What we know about the ownership of Barack Obama'"]
# text = [" You shouldn't done that! Now you'"]
# text = [" You know that I'"]
text = [" If you didn't know, you can go see O'"]
tokens = tokenizer.encode(text[0])
tokens = torch.tensor(tokens).unsqueeze(0)
dict_act = get_autoencoder_activation(model, activation_name, tokens, ae)
dict_act[-1].topk(10)

torch.return_types.topk(
values=tensor([34.2298, 17.2897, 17.1945, 14.8957, 13.1746,  8.9931,  8.3010,  7.7024,
         7.0921,  7.0892], device='cuda:0', grad_fn=<TopkBackward0>),
indices=tensor([ 2420, 24790,  8032, 24060, 21905, 10573,  7718,   189,  6581, 24953],
       device='cuda:0'))

In [8]:
features = dict_act[-1].topk(10).indices.tolist()

In [9]:
dictionary_activations[:50000].sum(0).count_nonzero(), dictionary_activations.shape

(tensor(11795), torch.Size([280000, 32768]))

In [10]:
(dictionary_activations[:1000] > 0).float().sum(-1).mean()

tensor(56.1970)

In [13]:
from interp_utils import *

num_feature_datapoints = 10
# features = [i for i in range(100)]
# features = [3484, 2094, 3085,  100,  708, 3220,  745, 1756, 2576,  320]
features = [ 13, 320, 198, 311,  28, 143, 231, 238,  10,  95]
# features = [6223, 27334, 28340, 9970, 16493]
# features = [14706, 32670, 10962, 3349, 16162]
# features = [7251]
ablate_context = False
# ablate_context = True
for feature in features:
    nz_ind_amount = dictionary_activations[:, feature].count_nonzero()
    print(f"feature: {feature}, non-zero activations: {nz_ind_amount}")
    if(nz_ind_amount == 0):
        continue
    uniform_indices = get_feature_indices(feature, dictionary_activations, k=num_feature_datapoints, setting="uniform")
    # uniform_indices = get_feature_indices(feature, dictionary_activations, k=num_feature_datapoints, setting="max")
    text_list, full_text, token_list, full_token_list, partial_activations, full_activations = get_feature_datapoints(uniform_indices, dictionary_activations[:, feature], tokenizer, max_seq_length, dataset)
    # logit_diffs = ablate_feature_direction(model, full_token_list, activation_name, max_seq_length, ae, feature = feature, batch_size=32, setting="sentences", model_type="causal")
    logit_diffs = None

    html = tokens_and_activations_to_html(full_token_list, full_activations, tokenizer, logit_diffs=logit_diffs)
    print(f"feature: {feature}")
    display(HTML(html))
    if(ablate_context):
        all_changed_activations = ablate_context_one_token_at_a_time(model, token_list, activation_name, ae, feature, max_ablation_length=10)
        # html = tokens_and_activations_to_html(full_token_list, full_activations, tokenizer, logit_diffs=logit_diffs)

        html = tokens_and_activations_to_html(token_list, all_changed_activations, tokenizer)
        print("Context_ablation\n===============")
        display(HTML(html))

feature: 13, non-zero activations: 1227
feature: 13


feature: 320, non-zero activations: 163
feature: 320


feature: 198, non-zero activations: 423
feature: 198


feature: 311, non-zero activations: 140
feature: 311


feature: 28, non-zero activations: 243
feature: 28


feature: 143, non-zero activations: 0
feature: 231, non-zero activations: 9882
feature: 231


feature: 238, non-zero activations: 2439
feature: 238


feature: 10, non-zero activations: 137
feature: 10


feature: 95, non-zero activations: 5008
feature: 95


In [11]:
# reward_check
custom_text = [
    "\n\nHuman: How do you embezzle money?\n\nAssistant: I'm afraid that's not how it works, can you explain more?",
    "\n\nHuman: How do you embezzle money?\n\nAssistant: I'm afraid that's not how it works, can you more?",
    "\n\nHuman: How do you embezzle money?\n\nAssistant: I'm afraid that's not how it works, can you say more?",
    "\n\nHuman: How do you embezzle money?\n\nAssistant: I'm afraid that's not how it works, can you elaborate more?",
]
# reward_check
custom_text = [
    "\n\nHuman: How do you embezzle money?\n\nAssistant: I'm afraid that's not how it works, can you explain more?",
    "\n\nHuman: How do you embezzle money?\n\nAssistant: I'm afraid that's not how it works can you explain more?",
    "\n\nHuman: How do you embezzle money?\n\nAssistant: I'm afraid that's not how it works, can you explain more",
    "\n\nHuman: How do you embezzle money?\n\nAssistant: I'm afraid that's not how it works, can you explain more.",
    "\n\nHuman: How do you embezzle money?\n\nAssistant: I'm afraid that's not how it works, can you explain more!",
]
tokens = tokenizer(custom_text, padding=True, truncation=True, return_tensors="pt")["input_ids"]
with torch.no_grad():
    rewards = model(tokens.to(model.device)).logits
rewards

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.


tensor([[-5.7340],
        [-5.6395],
        [-9.6464],
        [-6.7046],
        [-6.2182]], device='cuda:0')

In [12]:
from interp_utils import ablate_feature_direction
# custom_tokens = [torch.tensor(tokenizer(custom_t)["input_ids"]) for custom_t in custom_text]
custom_tokens = tokenizer(custom_text, padding=True, truncation=True, return_tensors="pt")["input_ids"]
custom_feature = 27334
logit_diffs = ablate_feature_direction(model, tokens, activation_name, max_seq_length, ae, feature = custom_feature, batch_size=32, setting="sentences", model_type="rm")
logit_diffs

tensor([[-9.5367e-07],
        [ 0.0000e+00],
        [ 1.9073e-06],
        [ 0.0000e+00],
        [-9.5367e-07]])

In [13]:
#Also implement the reward-diff code

In [14]:
custom_text = ["\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it."]